# Zap Imóveis Scraper

Este notebook é responsável por varrer o site zapimoveis.com.br em busca dos imóveis de interesse, para a criação de uma base de dados. A título de exemplo, este projeto se baseará apenas em casas disponíveis na cidade de Palmas - TO.

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

#### Quantos imóveis estão disponíveis? Em quantas páginas?

In [2]:
# Este projeto analisará casas à venda, disponíveis no site www.zapimoveis.com.br
url_base = 'https://www.zapimoveis.com.br/venda/casas/to+palmas/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

# Realizando o primeiro request e criando um soup
pagina_1 = BeautifulSoup(requests.get(url_base, headers=headers).text, 'html.parser')

# Quantos imóveis foram encontrados?
total_imoveis = str.split( pagina_1.find('header', 'summary').find('h1', 'js-summary-title').text )[0]
total_imoveis = int(str.replace(total_imoveis, '.', ''))

# Quantos imóveis aparecem na primeira página?
imoveis_por_pg = len(pagina_1.find('div', 'listings__container').findChildren("div", recursive=False))

# Quantas páginas no total?
total_paginas = int(np.ceil( total_imoveis/imoveis_por_pg ))
#total_paginas = 2

print(f'Total de Imóveis: {total_imoveis}. Por página: {imoveis_por_pg}. Páginas: {total_paginas}')

Total de Imóveis: 1483. Por página: 24. Páginas: 62


#### Função responsável por extrair todos os dados de interesse de uma página, imóvel a imóvel

In [3]:
def extrai_dados_da_pg(soup):
    
    imoveis = soup.find('div', 'listings__container')
    
    # Por algum problema do site, às vezes a última página de resultados aparece vazia, por isso essa precaução
    if imoveis == None:
        return ([],[],[],[],[],[])

    imoveis = imoveis.findChildren("div" , recursive=False)
    precos = []
    enderecos = []
    areas = []
    quartos = []
    garagens = []
    banheiros = []

    for imovel in imoveis:
        
        preco = str.replace( imovel.find('p', 'simple-card__price').text, '.', '' )
        preco = re.findall(r'[0-9]+', preco)[0] #ignora centavos, se tiver
        precos.append(int(preco))

        endereco = str.replace(imovel.find('h2', 'simple-card__address').text, '\n', '')
        enderecos.append(endereco.strip())

        area = re.search(r'[0-9]+', imovel.find('span', attrs={'itemprop':'floorSize'}).text).group(0)
        areas.append(int(area))

        quarto = imovel.find('span', attrs={'itemprop':'numberOfRooms'}).text
        quartos.append(int(quarto.strip()))

        garagem = imovel.find('li', 'js-parking-spaces')
        if garagem != None:
            garagem = int(garagem.findAll('span')[1].text.strip())
        else:
            garagem = 0
        garagens.append(garagem)

        banheiro = imovel.find('span', attrs={'itemprop':'numberOfBathroomsTotal'}).text
        banheiros.append(int(banheiro.strip()))

    return (precos, enderecos, areas, quartos, garagens, banheiros)

#### Realizando os requests de busca, página a página, e chamando a função 'extrai_dados_da_pg' para cada página

In [4]:
todos_precos = []
todos_enderecos = []
todas_areas = []
todos_quartos = []
todas_garagens = []
todos_banheiros = []

# Coletando os dados, página por página
for pag in range(1, total_paginas):
    soup = BeautifulSoup(requests.get(url_base + '?pagina=' + str(pag), headers=headers).text, 'html.parser')
    precos_pg, enderecos_pg, areas_pg, quartos_pg, garagens_pg, banheiros_pg = extrai_dados_da_pg(soup)
    todos_precos.extend(precos_pg)
    todos_enderecos.extend(enderecos_pg)
    todas_areas.extend(areas_pg)
    todos_quartos.extend(quartos_pg)
    todas_garagens.extend(garagens_pg)
    todos_banheiros.extend(banheiros_pg)

### Verificando se a extração ocorreu devidamente:

In [6]:
for i in range(3):
    print(f'IMÓVEL {i+1}:')
    print(f'-- Preço: R$ {todos_precos[i]}')
    print(f'-- Endereço: {todos_enderecos[i]}')
    print(f'-- Area: {todas_areas[i]}')
    print(f'-- Quartos: {todos_quartos[i]}')
    print(f'-- Garagens: {todas_garagens[i]}')
    print(f'-- Banheiros: {todos_banheiros[i]}')

print("Total preços: ", len(todos_precos))
menores_precos = np.sort(todos_precos)[:5]
maiores_precos = np.sort(todos_precos)[::-1][:5]
print("Menores Preços: ", menores_precos, "Maiores Preços: ", maiores_precos)

IMÓVEL 1:
-- Preço: R$ 1000000
-- Endereço: Plano Diretor Norte, Palmas
-- Area: 112
-- Quartos: 3
-- Garagens: 4
-- Banheiros: 2
IMÓVEL 2:
-- Preço: R$ 625000
-- Endereço: ARSO 43 Alameda 3, Plano Diretor Sul
-- Area: 150
-- Quartos: 3
-- Garagens: 2
-- Banheiros: 2
IMÓVEL 3:
-- Preço: R$ 315000
-- Endereço: Quadra ARNE 64 Alameda 2, Plano Diretor Norte
-- Area: 115
-- Quartos: 3
-- Garagens: 3
-- Banheiros: 1


#### Criamos um DataFrame para armazenar os dados coletados, fazemos simples conferências e gravamos em um arquivo .csv

In [8]:
imoveis_df = pd.DataFrame({
    'endereco': todos_enderecos,
    'preco': todos_precos,
    'area': todas_areas,
    'quartos': todos_quartos,
    'banheiros': todos_banheiros,
    'garagens': todas_garagens
})

imoveis_df.describe()

,preco,area,quartos,banheiros,garagens
count,1.455000e+03,1455.000000,1455.000000,1455.000000,1455.000000
mean,8.080155e+05,199.615120,3.104467,2.565636,2.326460
std,2.151745e+06,171.987841,1.019791,1.301645,1.466717
min,7.500000e+04,10.000000,1.000000,1.000000,0.000000
25%,4.500000e+05,120.000000,3.000000,2.000000,2.000000
50%,6.250000e+05,160.000000,3.000000,2.000000,2.000000
75%,8.500000e+05,238.000000,3.000000,3.000000,3.000000
max,8.000000e+07,4758.000000,32.000000,9.000000,19.000000


In [9]:
imoveis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1455 entries, 0 to 1454
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   endereco   1455 non-null   object
 1   preco      1455 non-null   int64 
 2   area       1455 non-null   int64 
 3   quartos    1455 non-null   int64 
 4   banheiros  1455 non-null   int64 
 5   garagens   1455 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 68.3+ KB


In [10]:
imoveis_df.to_csv('casas_zapimoveis.csv', index=False)